In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/parsed-annotations/parsed_annotations/11844559-result-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/6173686-background-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/53082542-method-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/11629674-result-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/10695055-background-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/5052952-method-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/8781666-result-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/2090262-result-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/1587-background-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/52194540-background-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/7898033-background-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/80628431-method-adju.json
/kaggle/input/parsed-annotations/parsed_annotations/5

In [118]:
import os
import math
import json
import torch
import torch.nn as nn

In [40]:
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', output_attentions=False, output_hidden_states=False)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/config.json
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b

In [126]:
def compute_rankings(ip_dir, file, max_length):
    input_dir = ip_dir
    filename = file
    query_facet = filename.split('-')[1]
    rankings = dict()
    max_length = max_length

    with open(input_dir + filename, 'r') as f:
        annot = json.load(f)
        query_dict = annot[0]
        candidate_dicts = annot[1:]
        candidate_relevances = [dict_['adju_relevance'] for dict_ in candidate_dicts]
        try:
            query_text = query_dict[query_facet + '_label']
            candi_texts = list()
            candi_encodings = list()
            candi_ratings = list()
            for candi_dict in candidate_dicts:
                try:
                    candi_texts.append(candi_dict[query_facet+'_label'])
                except:
                    candi_texts.append('')

            query_encoding = tokenizer.encode(query_text, padding='max_length')
            query_encoding = query_encoding + [tokenizer.pad_token_id] * (max_length - len(query_encoding))
            query_encoding = torch.tensor(query_encoding, dtype=torch.float32)
            csim = nn.CosineSimilarity(dim=0)

            for idx, text in enumerate(candi_texts):
                encoding = tokenizer.encode(text, padding='max_length')
                encoding = encoding + [tokenizer.pad_token_id] * (max_length - len(encoding))
                encoding = torch.tensor(encoding, dtype=torch.float32)
                candi_encodings.append(encoding)
                candi_ratings.append(csim(query_encoding, candi_encodings[idx]))

            sorted_indices = torch.argsort(torch.tensor(candi_ratings), descending=True)
            rankings = {i: int(docid) for i, docid in enumerate(sorted_indices)}
            candi_relevances = [candidate_relevances[i] for i in sorted_indices]
            return rankings, candi_relevances
        except:
            print("Facet : {}, Dict : {}".format(query_facet, query_dict.keys()))
            return None

In [133]:
input_dir = '/kaggle/input/parsed-annotations/parsed_annotations/'
max_length = 400

all_files  = os.listdir(input_dir)
filenames = [file for file in all_files if os.path.isfile(os.path.join(input_dir, file))]
results = dict()
for filename in filenames:
    paper_id = filename.split('-')[0]
    try:
        rankings, relevances = compute_rankings(input_dir, filename, max_length)
        results[paper_id] = {'rankings':rankings, 'relevances':relevances}
    except:
        results[paper_id] = None

Facet : result, Dict : dict_keys(['paper_id', 'title', 'background_label', 'abstract'])
Facet : result, Dict : dict_keys(['paper_id', 'title', 'background_label', 'abstract'])
Facet : method, Dict : dict_keys(['paper_id', 'title', 'background_label', 'abstract'])
Facet : background, Dict : dict_keys(['paper_id', 'title', 'background_label', 'method_label', 'result_label', 'abstract'])
Facet : method, Dict : dict_keys(['paper_id', 'title', 'background_label', 'abstract'])
Facet : method, Dict : dict_keys(['paper_id', 'title', 'background_label', 'abstract'])
Facet : method, Dict : dict_keys(['paper_id', 'title', 'background_label', 'abstract'])
Facet : method, Dict : dict_keys(['paper_id', 'title', 'background_label', 'abstract'])
Facet : background, Dict : dict_keys(['paper_id', 'title', 'method_label', 'result_label', 'abstract'])
Facet : method, Dict : dict_keys(['paper_id', 'title', 'background_label', 'abstract'])
Facet : background, Dict : dict_keys(['paper_id', 'title', 'method_l

In [134]:
with open('ranking_relevances.json', 'w') as f:
    json.dump(results, f)